In [1]:
import requests
from bs4 import BeautifulSoup

import json
import re

import time
import random

# Global sets to store unique ingredients and categories
all_ingredients = set()
all_categories = set()


# Dictionary mapping Unicode fraction symbols to their decimal equivalents
fraction_mapping = {
    '\u00BD': '1/5',  # 1/2
    '\u00BC': '1/4', # 1/4
    '\u00BE': '3/4', # 3/4
    '\u2153': '1/3', # 1/3
    '\u2154': '2/3', # 2/3
    '\u215b': '1/8', # 1/8
    '\u00b0': 'degrees ', #°
    '\u2013': '-', # -
    '\u00a0': ' ' # space
    # Add more mappings as needed
}

def replace_fraction_symbols(text):
    # Define a regex pattern that captures numbers possibly adjacent to fraction symbols
    pattern = r'(\d*)(%s)(\d*)' % '|'.join(re.escape(key) for key in fraction_mapping.keys())

    # Function to replace each match with appropriate spacing
    def replace(match):
        # Pre-number, fraction, and post-number
        pre, frac, post = match.groups()
        # Replace the fraction with its decimal equivalent from the dictionary
        frac_decimal = fraction_mapping[frac]
        # Add space if there is a preceding or succeeding number
        if frac_decimal == 'degrees ' or frac_decimal == '-' or frac_decimal == ' ':
            if pre and post:
                return f'{pre} {frac_decimal} {post}'
            elif pre:
                return f'{pre} {frac_decimal}'
            elif post:
                return f'{frac_decimal} {post}' 
        elif pre and post:
            return f'{pre} and {frac_decimal} {post}'
        elif pre:
            return f'{pre} and {frac_decimal}'
        elif post:
            return f'{frac_decimal} {post}'
        return frac_decimal

    return re.sub(pattern, replace, text)


## Extracts all recipe URLs from a given page. ##
def get_recipes_from_page(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Base URL for relative links
    base_url = "https://en.wikibooks.org"

    # Extract the href attributes, focusing on individual recipe pages
    recipe_links = []
    for link in soup.find_all('a', href=True):
        href = link['href']
        # Look for links that are likely to be individual recipes
        if href.startswith('/wiki/Cookbook:') and not "Category:" in href:
            full_url = base_url + href
            recipe_links.append(full_url)

    return recipe_links[2:-1]



def find_next_page_url(soup):
    # Assuming the "next page" link can be identified by specific text
    next_page_link = soup.find("a", text="next page")
    if next_page_link:
        return next_page_link['href']
    return None

def construct_full_url(relative_url):
    base_url = "https://en.wikibooks.org"
    return base_url + relative_url if relative_url else None

def generate_urls(base_url):
    next_page_urls = [base_url]
    response = requests.get(base_url)
    soup = BeautifulSoup(response.text, 'html.parser')
    next_page_relative_url = find_next_page_url(soup)
    while next_page_relative_url:
        full_url = construct_full_url(next_page_relative_url)
        next_page_urls.append(full_url)
        response = requests.get(full_url)  # Fetch next page content
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            next_page_relative_url = find_next_page_url(soup)
        else:
            print(f"Failed to load page {full_url}")
            break  # Break loop if the page fails to load
    return next_page_urls


## Scrapes detailed information about a recipe from its page. ##
def scrape_recipe_details(url):
    time.sleep(random.uniform(2.0, 4.0))  # Random delay between requests
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    response = requests.get(url)
            
    # Check if the request was successful
    if response.status_code == 200:
        # Parse the HTML content of the page with BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Attempting to extract data based on typical Wikibooks structure
        full_title = soup.title.text.replace(" - Wikibooks, open books for an open world", "").strip()
        # Use regex to find everything after "Cookbook:"
        match = re.search(r'Cookbook:(.*)', full_title)
        if match:
            title = match.group(1).strip()
        else:
            title = full_title  # Fallback to the full title if "Cookbook:" is not found
        difficulty = soup.find('table', class_='infobox')
        if difficulty:
            difficulty_row = difficulty.find('th', text=lambda text: text and 'Difficulty' in text)
            if difficulty_row:
                # The difficulty value might be represented by an image alt text
                difficulty_image = difficulty_row.find_next_sibling('td').find('img')
                if difficulty_image and 'alt' in difficulty_image.attrs:
                    difficulty = difficulty_image['alt'].strip()
                else:
                    difficulty = "Difficulty image or alt text not found."
            else:
                difficulty = "Difficulty row not found."
        else:
            difficulty = "Infobox table not found."

        servings_tag = soup.find('th', text='Servings')
        if servings_tag:
            servings_data = servings_tag.find_next_sibling('td')
            servings = servings_data.text if servings_data else 'Not specified'
        else:
            servings = 'Not specified'

        cooking_time = soup.find('table', class_='infobox')
        if cooking_time:
            time_row = cooking_time.find('th', string='Time')
            if time_row:
                time_data = time_row.find_next_sibling('td')
                if time_data:
                    time_text = time_data.text.strip()
                    # Use regex to capture everything after "Cooking:"
                    match = re.search(r"Cooking:\s*(.*)", time_text)
                    if match:
                        cooking_time = match.group(1).strip()  # Give only the text after "Cooking:"
                    else:
                        cooking_time = time_text
                else:
                    cooking_time = "Cooking time data not found."
            else:
                cooking_time = "Time row not found."
        else:
            cooking_time = "Infobox table not found."

        ingredients_tag = soup.find('span', text='Ingredients')
        if ingredients_tag and ingredients_tag.parent:
            ingredients_ul = ingredients_tag.parent.find_next_sibling('ul')
            if ingredients_ul:
                ingredients_list = [li.text.strip() for li in ingredients_ul.find_all('li')]
            else:
                ingredients_list = []
        else:
            ingredients_list = []

        directions_list = soup.find('ol')
        if directions_list:
            directions_list = [li.text.strip() for li in directions_list.find_all('li')]
        else:
            directions_list = ["Procedure section not found."]
        
        categories_elements = [a.text for a in soup.find_all('a', href=True) if 'Category:' in a['href']]
        
        procedure_span = soup.find('span', string='Procedure')
        notes_list = []
        if procedure_span and procedure_span.parent:
            notes_ul = procedure_span.parent.find_next_sibling('ul')
            if notes_ul:
                # Find all list item elements within the <ul> and extract their text
                notes_list = [li.text.strip() for li in notes_ul.find_all('li')]
            if not notes_list:  # If the list is empty, no notes were found
                notes_list = ['No notes available']
        else:
            notes_list = ['No procedure section found.']

    recipe_info = {
        'title': title,
        'level': difficulty,
        'yield': servings,
        'cooking time': replace_fraction_symbols(cooking_time),
        'ingredients': [replace_fraction_symbols(string) for string in ingredients_list],
        'steps': [replace_fraction_symbols(string) for string in directions_list],
        'cook note': [replace_fraction_symbols(string) for string in notes_list],
        'categories': categories_elements
    }
    
    all_ingredients.update(recipe_info['ingredients'])
    all_categories.update(recipe_info['categories'])
    
    return recipe_info


## Writes data to a JSON file. ##
def write_json(data, filename):
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(data, f, indent=4)


def main():
    counter = 0
    base_url = "https://en.wikibooks.org/wiki/Category:Recipes"
    urls = generate_urls(base_url)
    print(f"Found {len(urls)} urls.")
    print(f"{urls}")
    recipe_index = {}

    for url in urls:
        print(f"Processing {url}...")

        # Extract recipes from current page and scrape them
        recipes = get_recipes_from_page(url)
        print(f"Found {len(recipes)} recipes.")
        print(f"{recipes}")

        # Iterate over each URL to access and extract information
        for recipe in recipes:
            details = scrape_recipe_details(recipe)
            recipe_index[counter] = details
            counter += 1
            print(f"{recipe} has been processed.")
        print(f"{url} has been processed.")
    
    print("All URLs have been processed.")
    
    # Writing the index file and ingredient/category accumulations
    write_json(recipe_index, "recipes.json")
    write_json(list(all_ingredients), "all_ingredients.json")
    write_json(list(all_categories), "all_categories.json")

main() # TODO: UNCOMMENT TO RUN

/var/folders/vh/qxbx1gcd7xb1gcxc5z2l5nmw0000gn/T/ipykernel_16109/194200264.py:81: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  next_page_link = soup.find("a", text="next page")


Found 20 urls.
['https://en.wikibooks.org/wiki/Category:Recipes', 'https://en.wikibooks.org/w/index.php?title=Category:Recipes&pagefrom=Baked+Potato+Halves%0ABaked+Potato+Halves#mw-pages', 'https://en.wikibooks.org/w/index.php?title=Category:Recipes&pagefrom=Bonda+%28South+Indian+Vegetable+Fritter%29%0ABonda+%28South+Indian+Vegetable+Fritter%29#mw-pages', 'https://en.wikibooks.org/w/index.php?title=Category:Recipes&pagefrom=Champorado+%28Chocolate+Rice+Porridge%29%0AChamporado+%28Chocolate+Rice+Porridge%29#mw-pages', 'https://en.wikibooks.org/w/index.php?title=Category:Recipes&pagefrom=Chow+Mein%0AChow+Mein#mw-pages', 'https://en.wikibooks.org/w/index.php?title=Category:Recipes&pagefrom=Dalgona+Coffee%0ADalgona+Coffee#mw-pages', 'https://en.wikibooks.org/w/index.php?title=Category:Recipes&pagefrom=Fanke+%28Hausa+Puff+Puff%29%0AFanke+%28Hausa+Puff+Puff%29#mw-pages', 'https://en.wikibooks.org/w/index.php?title=Category:Recipes&pagefrom=Gin+Fried+Rice%0AGin+Fried+Rice#mw-pages', 'https://

/var/folders/vh/qxbx1gcd7xb1gcxc5z2l5nmw0000gn/T/ipykernel_16109/194200264.py:131: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  difficulty_row = difficulty.find('th', text=lambda text: text and 'Difficulty' in text)
/var/folders/vh/qxbx1gcd7xb1gcxc5z2l5nmw0000gn/T/ipykernel_16109/194200264.py:144: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  servings_tag = soup.find('th', text='Servings')
/var/folders/vh/qxbx1gcd7xb1gcxc5z2l5nmw0000gn/T/ipykernel_16109/194200264.py:171: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  ingredients_tag = soup.find('span', text='Ingredients')


https://en.wikibooks.org/wiki/Cookbook:20-Minute_Beef_Stroganoff has been processed.
https://en.wikibooks.org/wiki/Cookbook:40_Cloves_in_a_Roast_Chicken has been processed.
https://en.wikibooks.org/wiki/Cookbook:A_Nice_Cup_of_Tea has been processed.
https://en.wikibooks.org/wiki/Cookbook:Aadun_(Nigerian_Corn_Flour_with_Palm_Oil) has been processed.
https://en.wikibooks.org/wiki/Cookbook:Abacha_Mmiri_(Soaked_Cassava_Flakes) has been processed.
https://en.wikibooks.org/wiki/Cookbook:Abak_Soup has been processed.
https://en.wikibooks.org/wiki/Cookbook:Abari_(Nigerian_Steamed_Corn_Pudding) has been processed.
https://en.wikibooks.org/wiki/Cookbook:Abula_(Nigerian_Three_Stews) has been processed.
https://en.wikibooks.org/wiki/Cookbook:Accra_Cassava_(Cameroonian_Cassava_Fritters) has been processed.
https://en.wikibooks.org/wiki/Cookbook:Achi_Soup has been processed.
https://en.wikibooks.org/wiki/Cookbook:Achicha_with_Fio_Fio_(Pigeon_Peas_and_Taro) has been processed.
https://en.wikibooks.or